# New Data Preprocessing

## Import

In [1]:
import pandas as pd

## Hyper Parameter

In [2]:
# Local
DATA_DIR = '/Users/inseoklee/Desktop/miso/data/aihub_sentiment'
SAVE_DIR = '/Users/inseoklee/Desktop/miso/data'

## Load Data

In [3]:
df6 = pd.read_excel(DATA_DIR+'/senti_training.xlsx')
df7 = pd.read_excel(DATA_DIR+'/senti_validation.xlsx')

In [4]:
df = pd.concat([df6, df7], ignore_index=True)
df.shape

(46009, 15)

In [5]:
df[:1]

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,44164,청년,남성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN


## Data PreProcessing

### remove unnecessary columns

In [6]:
try:
    df['first_text'] = df['사람문장1']
    df['second_text'] = df['사람문장2']
    df['label'] = df['감정_대분류']
    df = df[['first_text', 'second_text', 'label']]
except:
    print("Already Preprocessed")

In [7]:
df[:5]

,first_text,second_text,label
0,아내가 드디어 출산하게 되어서 정말 신이 나.,아 지금 정말 신이 나.,기쁨
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,불안
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,당황
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,기쁨
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,기쁨


### 두개의 데이터 프레임으로 나눈 후 다시 합치기

In [8]:
first_df = df[['first_text', 'label']]
second_df = df[['second_text', 'label']]

In [9]:
first_df[:2]

,first_text,label
0,아내가 드디어 출산하게 되어서 정말 신이 나.,기쁨
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,불안


In [10]:
first_df.rename(columns={'first_text':'text'}, inplace=True)
second_df.rename(columns={'second_text':'text'}, inplace=True)

/var/folders/vs/v6rsnrsn6fj5d0n5cvfkwqnr0000gn/T/ipykernel_18907/733090216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_df.rename(columns={'first_text':'text'}, inplace=True)
/var/folders/vs/v6rsnrsn6fj5d0n5cvfkwqnr0000gn/T/ipykernel_18907/733090216.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_df.rename(columns={'second_text':'text'}, inplace=True)


In [11]:
concat_df = pd.concat([first_df, second_df], ignore_index=True)

In [12]:
concat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92018 entries, 0 to 92017
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    92018 non-null  object
 1   label   92018 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


### Drop duplicates

In [13]:
concat_df['text'].nunique()

87682

In [14]:
concat_df.drop_duplicates(subset=['text'], inplace=True)

### Remove missing value

In [15]:
concat_df['label'].unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처', '불안 ', '기쁨 ', 's'],
      dtype=object)

In [16]:
concat_df.drop(concat_df[concat_df['label'] == 's'].index, inplace=True)

In [17]:
concat_df['label'].unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처', '불안 ', '기쁨 '], dtype=object)

### Labeling

In [18]:
emo_dic = {
    '기쁨': 'happy', '기쁨 ': 'happy', '불안': 'anxious', '불안 ': 'anxious', '당황': 'embarrassed', '슬픔': 'sad', '분노': 'angry', '상처': 'hurt'
}

In [19]:
concat_df['label'] = concat_df['label'].replace(emo_dic)
concat_df['label'].unique()

array(['happy', 'anxious', 'embarrassed', 'sad', 'angry', 'hurt'],
      dtype=object)

## Save Data

In [20]:
concat_df.to_csv(SAVE_DIR + '/new_processed_sentiment.csv', encoding="utf-8-sig", index=False)